Le vecteur est définit dans rotation.py
On utilise matplotlib pour le dessiner.

In [1]:
%matplotlib notebook
import numpy as np
import ipywidgets as widgets

class point:
    def __init__(self, l = [0, 0 ,0]):
        self.x = l[0]
        self.y = l[1]
        self.z = l[2]
        self.comp = l

    def __getitem__(self, key):
        if key < 0 or key > 2:
            return None
        else:
            return self.comp[key]

    def __iter__(self):
        return iter(self.comp)

class vector:
    def __init__(self, dest = [0, 0, 0], orig = [0, 0, 0]):
        self.orig = point(orig)
        self.dest = point(dest)
        self.comp = point([dest[i] - orig[i] for i in range(3)])
        self.norm = np.sqrt(sum(c**2 for c in self.comp))

    def __iter__(self):
        return iter(self.comp)

    def __getitem__(self, key):
        return self.comp[key]

    def __mul__(self, x = 1):
        return vector([c*x for c in self.comp], self.orig)
    def __div__(self, x = 1):
        return vector([c/x for c in self.comp], self.orig)
    def __add__(self, m = [0, 0, 0]):
        return vector([self[i]+m[i] for i in range(3)], self.orig)
    def __sub__(self, m = [0, 0 ,0]):
        return vector([self[i]-m[i] for i in range(3)], self.orig)

    def show(self):
        print("[{} {} {}]".format(self[0], self[1], self[2]))
        return self

    def unit(self):
        return vector([c/self.norm for c in self.comp], self.orig)

    def angle(self, v):
        return np.arccos(self.dot_product(v)/(self.norm*v.norm))

    def dot_product(self, v):
        return sum(self[i]*v[i] for i in range(3))

    def cross_product(self, v):
        a = self[1]*v[2] - self[2]*v[1]
        b = self[2]*v[0] - self[0]*v[2]
        c = self[0]*v[1] - self[1]*v[0]
        return vector([a, b, c])

    def projection(self, v):
        return v*(self.dot_product(v)/(v.norm**2))

    def rotate(self, dir, angle, degrees = True):
        th = angle*(np.pi/180) if degrees else angle
        u = dir.unit()
        p = quaternion(np.cos(th/2), u*np.sin(th/2))
        q = quaternion(0, self)
        return (p*q*p.inverse()).imag

class quaternion:
    def __init__(self, a = 0, v: vector = vector()):
        self.real = a
        self.imag = v

    def __mul__(self, q):
        a, u = self.real, self.imag
        b, v = q.real, q.imag
        c = a*b - u.dot_product(v)
        w = v*a + u*b + u.cross_product(v)
        return quaternion(c, w)

    def __add__(self, q):
        return quaternion(self.real + q.real, self.imag + q.imag)

    def __sub__(self, q):
        return quaternion(self.real - q.real, self.imag - q.imag)

    def show(self):
        print("({}, {}, {}, {})".format(self.real, self.imag[0], self.imag[1], self.imag[2]))
        return self

    def inverse(self):
        return quaternion(self.real, vector() - self.imag)
    
def xunit(v: vector = vector()):
        return vector([1.0, 0, 0], v.orig)
def yunit(v: vector = vector()):
    return vector([0, 1.0, 0], v.orig)
def zunit(v: vector = vector()):
    return vector([0, 0, 1.0], v.orig)


def make_angle_slider(self):
    return widgets.IntSlider(min = 0, max = 360, step = 1)

class scene:
    def __init__(self, v, d = xunit()):
        self.fig = plt.figure()
        self.ax = self.fig.add_subplot(111, projection = "3d", xlim = (-3, 3), ylim = (-3, 3), zlim = (-3, 3))

        self.vangle = 0
        self.dangle = 0
        self.vi = self.v = v
        self.di = self.d = d
        self.Qv = self.quiver(v)
        self.Qd = self.quiver(d, "green")

        self.xu = xunit(self.v)
        self.yu = yunit(self.v)
        self.zu = zunit(self.v)

        plt.ion()
        plt.show()

    def make_angle_slider(self):
        return widgets.IntSlider(min = 0, max = 360, step = 1)

    def quiver(self, v: vector, col = "black"):
        o = v.orig
        d = v.dest
        q = self.ax.quiver(o[0], o[1], o[2], d[0], d[1], d[2], color = col)
        return q

    def rotate_vector(self, angle_vecteur):
        self.vangle = angle_vecteur
        self.v = self.vi.rotate(self.d, self.vangle, True)
        self.ax.collections.remove(self.Qv)
        self.Qv = self.quiver(self.v)
        self.fig.canvas.draw()

    def rotate_dir(self, d, angle_direction):
        self.dangle = angle_direction
        self.d = self.di.rotate(d, self.dangle, True)
        self.vi = self.v.rotate(self.d, -self.vangle, True)
        self.ax.collections.remove(self.Qd)
        self.Qd = self.quiver(self.d, "green")
        self.fig.canvas.draw()

    def rotate_3D(self, angleX, angleY, angleZ):
        self.vangle = 0
        self.v = self.vi.rotate(self.xu, angleX, True) \
            .rotate(self.yu, angleY, True) \
            .rotate(self.zu, angleZ, True)
        self.ax.collections.remove(self.Qv)
        self.Qv = self.quiver(self.v)
        self.fig.canvas.draw()

Commençons par tracer le vecteur V = \[2 2 2\] dans l'espace:

In [2]:
# On crée le vecteur V qui part de l'origine comme [2 2 2]
V = vector([2.0, 2.0, 2.0])

# On crée une scène pour cette démonstration en utilisant le vecteur V
s1 = scene(V)

NameError: name 'plt' is not defined

Faisons-le tourner par rapport à l'axe des x

In [ ]:
# Production du slider qui définit l'angle du vecteur par rapport au vecteur unitaire en x
widgets.interact(s1.rotate_vector, angle_vecteur = s1.make_angle_slider())

Si on utilise un vecteur unitaire d vers z comme axe de rotation et qu'on le fait pivoter par rapport à l'axe des x, nous pouvons faire tourner le vecteur dans toutes les directions de l'espace.

In [ ]:
# On crée un vecteur directionel qui sera variable (arbitrairement vers les z)
d = vector(zunit(V))

# On crée la scène pour cette démonstration, en ajoutant le vecteur d variable
s2 = scene(V, d)

# On produit deux sliders pour définir l'angle de rotation de V autour de d,
# et un angle pour définit l'angle de d autour de l'axe des x
widgets.interact(s2.rotate_vector, angle_vecteur = s2.make_angle_slider())
widgets.interact(s2.rotate_dir, d = widgets.fixed(xunit()), angle_direction = s2.make_angle_slider())

On peut rajouter des axes en y et z pour le faire tourner en 3D d'une façon plus intuitive

In [ ]:
# On crée une nouvelle scène pour la démonstration
s3 = scene(V)

# On trace les vecteurs unitaires en x, en y et en z
s3.quiver(xunit(V), "red")
s3.quiver(yunit(V), "blue")
s3.quiver(zunit(V), "green")

# On trace trois sliders pour définir l'angle autour des vecteurs unitaires en x, y et z
widgets.interact(s3.rotate_3D,
                 v = widgets.fixed(V),
                 angleX = s3.make_angle_slider(),
                 angleY = s3.make_angle_slider(),
                 angleZ = s3.make_angle_slider())